In [1]:
from backtest import *
from config import *
from strategies import *
import json
import pandas as pd
from sqlalchemy import create_engine
from typing import List, NoReturn
import os

/Users/arsenchik/miniforge3/envs/algotrading/lib/python3.11/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
res_dir_path = '/Users/arsenchik/Desktop/dipploma/machine_learning_in_hft/algotrading/results'
exploring_info = {}

In [3]:
for part_name in os.listdir(res_dir_path):
    time_frame = part_name[4:]
    full_part_path = os.path.join(res_dir_path, f'{part_name}/top_5.json')
    df = pd.read_json(full_part_path)
    exploring_info[time_frame] = list(df.index)

In [4]:
exploring_info

{'5m': ['BTCUSDT', 'ETHUSDT', 'MATICUSDT', 'SANDUSDT', 'SOLUSDT'],
 '1m': ['BTCUSDT', 'ETHUSDT', 'MATICUSDT', 'SANDUSDT', 'SOLUSDT'],
 '3m': ['BTCUSDT', 'ETHUSDT', 'MATICUSDT', 'SANDUSDT', 'SOLUSDT'],
 '1h': ['BTCUSDT', 'ETHUSDT', 'MATICUSDT', 'SANDUSDT', 'SOLUSDT'],
 '15m': ['BTCUSDT', 'ETHUSDT', 'MATICUSDT', 'SANDUSDT', 'SOLUSDT'],
 '30m': ['BTCUSDT', 'ETHUSDT', 'MATICUSDT', 'SANDUSDT', 'SOLUSDT']}

In [17]:
def validation(
    pairs: List,
    strategy: type[Strategy],
    url: str,
    time_frame: str,
    td_days: int,
    cash: float = 10000,
    commission: float = 0.001,
) -> NoReturn:
    engine = create_engine(url, pool_size=10, max_overflow=20)
    with engine.connect() as connection:
        for pair in pairs:
            qry = f"SELECT * FROM '{pair}' WHERE Open_Time >= '{pd.to_datetime('today') - timedelta(days = td_days)}'"
            data = pd.DataFrame(connection.execute(text(qry))).set_index("Open_Time")
            data.index = pd.to_datetime(data.index)

            bt = Backtest(
                data, strategy, cash=cash, commission=commission, exclusive_orders=True
            )
            output = bt.run()
            print(f'Current {pair} and current {time_frame}:')
            # output = pd.concat(
            #     [pd.Series([symbol, time_frame], ["Pair", "TimeFrame"]), output]
            # )
            # full_part_output_path = os.path.join(
            #     full_dir_output_path, f"res_{symbol}.json"
            # )
            # output.to_json(full_part_output_path, date_format="iso")
            # returns.append(output["Return [%]"])
            bt.plot()
    connection.close()
        

In [18]:
for time_frame in exploring_info:
    # print(time_frame)
    print(time_frame_to_data_base[time_frame])
    validation(exploring_info[time_frame], SmaCross, time_frame_to_data_base[time_frame], time_frame, 3)

sqlite:///database/Crypto_5m.sqlite


OperationalError: (sqlite3.OperationalError) unable to open database file
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [19]:
engine = create_engine(time_frame_to_data_base[time_frame], echo=True)
enigne.execute(text("SELECT * "))

OperationalError: (sqlite3.OperationalError) unable to open database file
(Background on this error at: https://sqlalche.me/e/20/e3q8)